Changes from 1.0

- simpler architecture: lstm of feature sequences directly to target sequences
  - similar to [Creating A Text Generator Using Recurrent Neural Network ](https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/)
- min/max scaling be per series
- model's "subtract" output MAE with target = zeros changed to "binary cross-entropy"

- check if better to just use multi-core with smaller batch-size ~ 32 than GPU
    - indeed it is
    - batch size 16*1024 .. Epoch 33/400    - 7s - loss: 0.6803 - val_loss: 0.6792
    - batch size     512
      - Epoch 1/400    - 22s - loss: 0.6700 - val_loss: 0.6361
      - Epoch 2/400    - 21s - loss: 0.6558 - val_loss: 0.6312
    - note that `batch_size=32` in benchmarks [here](https://www.tensorflow.org/performance/performance_guide#optimizing_for_cpu)


- is downsampling harmful for training/prediction?
  - downsample_pts = 1 is very very slow
- Add AveragePooling1D layer as base (10x downsampling)
  - validation loss was horrible
- ~~instead of striding, use non-overlapping windows~~
  - turns out that eventhough the training was good, the predictions were not
  - maybe because lstm requires that input not be shuffled
  - another point was how I increment the dataset to have balanced classes of HandStart=0 and HandStart=1
  - TODO train the LSTM with balanced classes, no overlap, and without shuffling
  
- TODO adding Conv1D ahead of LSTM
- TODO also interesting is [ConvLSTM2D](https://keras.io/layers/recurrent/#convlstm2d)
  - and the [keras example](https://github.com/keras-team/keras/blob/ce4947cbaf380589a63def4cc6eb3e460c41254f/examples/conv_lstm.py)
  
- TODO lahead is currently in "points". So needs to be changed depending on downsampling.

## check gpu usage

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## ~~set multi-core~~

- [ref 0](https://stackoverflow.com/a/49530131/4126114)
- [ref 1](https://datascience.stackexchange.com/a/22840/35596)
- [ref 2](https://stackoverflow.com/a/45843766/4126114)

Didn't help ... on c5.18xlarge still using 10 cores maximum, even if 72 are available

## some parameters

In [ ]:
# n_train: number of points for training, as opposed to testing
# lahead: stride data with "lahead" window size
# batch_size: keras.model.fit parameter .. smaller batches lead to less loss of data when truncating non-multiples of batch_size
# downsample_pts: 1 for no downsampling, 10 for downsample by 10
#---------------------------------------------------------
# set 1
# n_train, lahead, batch_size, downsample_pts = 120000, 10, 2**14, 10

# set 2
# n_train, lahead, batch_size, downsample_pts = 1200000, 100, (2**10)*(2**8), 1 # batch_size = 1024
# n_train, lahead, batch_size, downsample_pts = 1200000, 100, 2**8, 1 # batch_size = 256

# note that lahead=150 matches perfectly with non-downsampled length of HandStart=1 length
# note smaller batch_size since non-overlap causes smaller number of HanStart=1 samples
n_train, lahead, batch_size, downsample_pts = 1200000, 150, 2**5, 1 # batch_size = 32

# set 3:
# training each subject / series separately
# Requires smaller batch_size since each series is only around 1000 pts when downsampled by 10
# n_train, lahead, batch_size, downsample_pts = 120000, 10, 2**4, 10

## import libraries

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import time

# https://keras.io/layers/recurrent/#lstm
from keras.models import Sequential, Model
from keras.layers import (Dense, LSTM, Lambda, Dropout, Embedding, Flatten,
                         Subtract, Dot, Activation,
                         Input, RepeatVector, TimeDistributed, Concatenate,
                         Conv1D, MaxPooling1D, AveragePooling1D
                         )

import numpy as np

from sklearn.preprocessing import MinMaxScaler

## load data

In [ ]:
def my_load(subj_ids:list, series_ids:list):
    features_all = []
    targets_all = []
    for i1 in subj_ids:
        for i2 in series_ids:
            for i3, fn in [
                ('features', 'data/raw/train/subj%i_series%i_data.csv'%(i1, i2)),
                ('targets', 'data/raw/train/subj%i_series%i_events.csv'%(i1, i2)),
            ]:
                print('status', i1, i2, i3)
                xxx_i = pd.read_csv(fn)
                xxx_i['subj_id'] = i1
                xxx_i['series_id'] = i2
                xxx_i = xxx_i.set_index(['subj_id', 'series_id', 'id']).astype('int16')
                xxx_i = xxx_i[::downsample_pts] # downsample
                if i3=='features':
                    features_all.append(xxx_i)
                else:
                    targets_all.append(xxx_i)
            
    features_all = pd.concat(features_all, axis=0)
    targets_all = pd.concat(targets_all, axis=0)
    return features_all, targets_all

In [ ]:
train_features, train_targets = my_load(subj_ids = [1], series_ids = [x+1 for x in range(8)])
train_features.shape, train_targets.shape

In [ ]:
train_features.head(n=2)

In [ ]:
train_targets.head(n=2)

## preprocess features

e.g. scale to [0,1], stride, truncate, etc

In [ ]:
def stride_df(df, n_back):
    """
    create rolling windows for LSTM
    """
    out = []
    for i in range(n_back):
        out.append(df.shift(i).values)
        
    out = np.stack(out, axis=2)[(n_back-1):, :, :] # drop first lahead
    out = np.swapaxes(out, 1, 2)
    out = np.flip(out, axis=1) # so that the index=0 is the oldest, and index=4 is latest
    return out


def nonstride_df(a:np.array, n_back):
    """
    create non-rolling windows (numpy)
    """
    # truncate non-multiples of n_back
    to_drop = a.shape[0] % n_back
    print('nonstride_df, a.shape[0], to_drop', a.shape[0], to_drop)
    a = a[to_drop:]
    return a.reshape((-1,n_back,a.shape[1]))


stride_df_20 = lambda df: stride_df   (df       , lahead)
stride_df_21 = lambda df: nonstride_df(df.values, lahead)
stride_df_22 = lambda  a: nonstride_df(a,         lahead)

In [ ]:
def my_truncate(df):
    """
    drop 1st x rows if they are not a multiple of batch_size
    """
    return df.tail(df.shape[0] - (df.shape[0]%batch_size))

In [ ]:
def wrap_pd_df(xxx, func):
    return pd.DataFrame(
             func(xxx), 
             columns=xxx.columns, 
             index=xxx.index
           )

def preprocess(x_train, y_train, with_overlap):
    scaler = MinMaxScaler()
    
    print('min/max start')
    # xtrain_pre = x_train.groupby(['subj_id', 'series_id']).apply(lambda xxx: scaler.fit_transform(xxx))
    xtrain_pre = ( x_train.groupby(['subj_id', 'series_id'])
                          .apply(lambda xxx: wrap_pd_df(xxx, lambda yyy: scaler.fit_transform(yyy)))
                 )
    ytrain_pre = y_train # just a copy since no scaling done

    print('train_pre', xtrain_pre.shape, ytrain_pre.shape)
    #--------------------------------------
    # FIXME striding can be parallelized
    
    stride_func = stride_df_20 if with_overlap else stride_df_21
        
    
    # xtrain_roll = stride_df_2(xtrain_pre)
    # ytrain_roll = stride_df_2(ytrain_pre)
    xtrain_roll = (xtrain_pre.groupby(['subj_id', 'series_id'])
                             .apply(stride_func)
                             # .apply(lambda xxx: wrap_pd_df(xxx, stride_df_2))
                  )
    ytrain_roll = (ytrain_pre.groupby(['subj_id', 'series_id'])
                             .apply(stride_func)
                             # .apply(lambda xxx: wrap_pd_df(xxx, stride_df_2))
                  )

    # "meta" dataframe that will still contain the pandas index (above *_roll variables are numpy matrices)
    # ztrain_roll = y_train.groupby(['subj_id', 'series_id']).apply(lambda group: group.iloc[(lahead-1):])
    if with_overlap:
        ztrain_roll = y_train.groupby(by=['subj_id', 'series_id']).apply(lambda group: group.iloc[(lahead-1):].reset_index(drop=True))
    else:
        ztrain_roll = stride_df_22(ytrain_pre.iloc[:,0].reset_index().index.values.reshape((-1,1)))
        ztrain_roll = y_train.iloc[ztrain_roll[:,-1,0]]


    print('train_roll 1', xtrain_roll.shape, ytrain_roll.shape, ztrain_roll.shape)
    #return xtrain_roll, ytrain_roll, ztrain_roll

    """
    # drop non-batchsize-multiple per subject/series pair
    for (subj_id, series_id), group in xtrain_roll.groupby(['subj_id', 'series_id']):
        to_drop = group.values[0].shape[0] % batch_size
        print(subj_id, series_id, 'drop non-multiple', to_drop)
        assert to_drop < 1000

        xtrain_roll.loc[subj_id, series_id] = xtrain_roll.loc[subj_id, series_id][(to_drop):]
        ytrain_roll.loc[subj_id, series_id] = ytrain_roll.loc[subj_id, series_id][(to_drop):]
       
    ztrain_roll = ztrain_roll.groupby(['subj_id', 'series_id']).apply(my_truncate)
    print('train_roll 2', xtrain_roll.shape, ytrain_roll.shape, ztrain_roll.shape)
    """
    
    # aggregate all strided matrices (since done per subj_id and series_id)
    xtrain_roll = np.concatenate(xtrain_roll.values, axis=0)
    ytrain_roll = np.concatenate(ytrain_roll.values, axis=0)
    
    # drop non-batchsize-multiple, once for all
    to_drop = xtrain_roll.shape[0] % batch_size
    print('drop non-multiple of batch_size', to_drop)
    xtrain_roll = xtrain_roll[(to_drop):]
    ytrain_roll = ytrain_roll[(to_drop):]
    ztrain_roll = my_truncate(ztrain_roll)
    print('train_roll 2', xtrain_roll.shape, ytrain_roll.shape, ztrain_roll.shape)
    
    assert xtrain_roll.shape[0]>0, "lost all data ... batch_size=%s is too high"%batch_size
    
    return xtrain_roll, ytrain_roll, ztrain_roll

In [ ]:
x_train = train_features.head(n=n_train).copy()
y_train = train_targets.head(n=n_train).copy()
print('x_train, y_train', x_train.shape, y_train.shape)

In [ ]:
# calculate ratio of HandStart = 0 to = 1 to get the target class imbalance
ratio_0_1 = y_train.groupby('HandStart').size()
print(ratio_0_1)
ratio_0_1 = ratio_0_1.loc[0] // ratio_0_1.loc[1]
ratio_0_1

## Identify each HandStart=1

In [ ]:
# calculate length of HandStart == 1
y_cols = y_train.columns
for k in y_cols: # e.g. 'HandStart'
    y_temp1 = y_train[k].diff().fillna(value=0)
    y_temp2 = y_temp1.copy()
    y_temp2[y_temp2 < 0] = 0
    y_temp2 = y_temp2.cumsum()
    y_train['%s_id'%k] = y_train[k] * y_temp2

y_train[[x for x in y_train.columns if x.endswith('_id')]].head(n=10000).plot()
plt.show()

In [ ]:
# all HandStarts are of length 150
assert set(y_train[y_train['HandStart_id']>0].groupby('HandStart_id').size()) == set([150])

In [ ]:
set(y_train[y_train['LiftOff_id']>0].groupby('LiftOff_id').size())

In [ ]:
y_train.shape

## Create separate non-overlapping windows 

In [ ]:
# Create separate non-overlapping windows from HandStart=1 and =0, and then concatenate
# This way, we don't get a window half of which has HandStart=0 and the other half = 1
xtrain_roll_1, ytrain_roll_1, ztrain_roll_1 = preprocess(x_train[y_train['HandStart']==1], y_train[y_train['HandStart']==1], with_overlap=False)
xtrain_roll_0, ytrain_roll_0, ztrain_roll_0 = preprocess(x_train[y_train['HandStart']==0], y_train[y_train['HandStart']==0], with_overlap=False)

# repeat the *_1 40x times to balance against the *_0 class (check above for how to calculate 40)
xtrain_roll_1 = np.repeat(xtrain_roll_1, repeats=ratio_0_1, axis=0)
ytrain_roll_1 = np.repeat(ytrain_roll_1, repeats=ratio_0_1, axis=0)

z_np = np.repeat(ztrain_roll_1.values, repeats=ratio_0_1, axis=0)
z_cols = ztrain_roll_1.columns
z_inds = ztrain_roll_1.index
print('z shape', ztrain_roll_1.shape, z_np.shape, z_cols.shape, z_inds.shape)
ztrain_roll_1 = pd.DataFrame(
    z_np,
    columns = z_cols,
    index = z_inds.repeat(ratio_0_1)
)

# concatenate _0 with _1
xtrain_roll = np.concatenate([xtrain_roll_1, xtrain_roll_0])
ytrain_roll = np.concatenate([ytrain_roll_1, ytrain_roll_0])
ztrain_roll = pd.concat([ztrain_roll_1, ztrain_roll_0])

assert xtrain_roll.shape[0] > 0
xtrain_roll.shape, ytrain_roll.shape, ztrain_roll.shape

In [ ]:
197%32

In [ ]:
x_train.head(n=2)

In [ ]:
x_train[['Fp1', 'Fp2']].plot(figsize=(20,3), alpha=0.5)
plt.show()

In [ ]:
y_train[['HandStart']].head(n=10000).plot(figsize=(20,3), alpha=0.5)
plt.show()

In [ ]:
# Plotting ":,-1,:" below is like plotting every "lahead" points
xtrain_plot = pd.DataFrame(xtrain_roll[:,-1,:], columns=x_train.columns)
print(xtrain_plot.shape)
xtrain_plot[['Fp1', 'Fp2']].plot(figsize=(20,3), alpha=0.5)
# plt.title('subj_id=1, series_id=1')
plt.show()

In [ ]:
# Plot below should have a balanced number of points for 1 and 0
# if the 1's are repeated enough
#
# Plotting ":,-1,:" below is like plotting every "lahead" points

ytrain_plot = pd.DataFrame(ytrain_roll[:,-1,:], columns=y_train.columns)
print(ytrain_plot.shape)
ytrain_plot['HandStart'].plot(figsize=(20,3), alpha=0.5)
# plt.title('subj_id=1, series_id=1')
plt.show()

In [ ]:
# below should show {0,100} showing that all axis=1 dim is flat
set(ytrain_roll[:,:,0].sum(axis=1))

## shuffle the data

In [ ]:
new_ind = np.arange(xtrain_roll.shape[0])
np.random.shuffle(new_ind)
xtrain_roll = xtrain_roll[new_ind]
ytrain_roll = ytrain_roll[new_ind]
ztrain_roll = ztrain_roll.iloc[new_ind]
xtrain_roll.shape, ytrain_roll.shape, ztrain_roll.shape

## fit model: AE coupled with regression on target

In [ ]:
def create_coupled():
    lstm_dim_1 = 15
    len_feat = xtrain_roll.shape[2]
    len_targ = 1
    input_shape = (lahead, len_feat, )

    # input
    feat_raw = Input(shape=input_shape, name='raw_features')
    
    # downsample
    feat_conv = feat_raw
    # feat_conv = AveragePooling1D(pool_size = 10)(feat_conv)

    # convolve and maxpool
    # feat_conv = Conv1D(filters = 32, kernel_size = 5, padding='valid', activation='relu', strides=1)(feat_conv)
    # feat_conv = MaxPooling1D(pool_size = 4)(feat_conv)

    # features encoder
    feat_enc = feat_conv
    feat_enc = LSTM(
              lstm_dim_1,
              batch_size=batch_size,
              return_sequences=False,
              activation='tanh',
              name='encoded_features')(feat_enc)

    # features decoder
    targ_rec = feat_enc
    targ_rec = RepeatVector(lahead, input_shape=(lstm_dim_1, ))(targ_rec)
    targ_rec = LSTM(lstm_dim_1,
              batch_size=batch_size,
              return_sequences=True,
              dropout=0.2,
              activation='tanh')(targ_rec)
    targ_rec = TimeDistributed(
        # Dense(len_targ, activation='linear'),
        Dense(len_targ, activation='sigmoid'),
        name='reconstructed_targets'
    )(targ_rec)

    # create model
    model_all = Model(inputs = [feat_raw], outputs = [targ_rec])
    return model_all

In [ ]:
from keras import backend as K
from keras.losses import binary_crossentropy
def double_binary_crossentropy(y_true, y_pred):
    return K.mean(binary_crossentropy(y_true, y_pred), axis=-1)


mod2 = create_coupled()
# mod2.compile(loss='mae', optimizer='adam')
mod2.compile(loss=double_binary_crossentropy, optimizer='adam')
# mod2.compile(loss=double_binary_crossentropy, optimizer='adam', sample_weight_mode="temporal")
mod2.summary()

In [ ]:
def my_predict(model, np_in, index):
    
    # make prediction
    targ_rec = model.predict(np_in, batch_size=batch_size)
        
    # plot target reconstruction
    feat_int = 0
    pd.DataFrame({
        'actual': pd.Series(np_in['raw_targets'][:,-1,feat_int],  index=index).astype('int16'),
        'pred': pd.Series(targ_rec[:,-1,feat_int],  index=index),
    }).plot(figsize=(20,3), alpha=0.5)
    plt.title('target %i'%(feat_int))
    plt.legend()
    plt.show()
    
    # prepare output
    out = pd.DataFrame({
        'prediction': targ_rec[:,-1,0].squeeze(), 
        'id': index,
    }).set_index(['id'])
    return out

In [ ]:
print(time.ctime(),'fit start')
history = mod2.fit(
         {   'raw_features': xtrain_roll,
         },
         {   'reconstructed_targets': ytrain_roll[:,:,:1],
         },
         batch_size=batch_size,
         epochs=100,
         # initial_epoch = 17,
         verbose=2,
         #validation_data=None,
         validation_split = 0.3,
         shuffle=False
    )
print(time.ctime(),'fit end')

In [ ]:
# ignore first few points since large relative to others
# plt.plot(history.history['loss'][5:], label='loss')
plt.plot(history.history['loss'], label='loss') # [5:]
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('training loss')
plt.show()

In [ ]:
# predict on whole series (plots implicitly actual vs predicted)
n_show = 100
for n_start in [0,  50, 100, 150]:
    ytrain_pred = my_predict(
        mod2,
        {   'raw_features': xtrain_roll[n_start:(n_start+n_show)],
            'raw_targets':  ytrain_roll[n_start:(n_start+n_show)] + 1,
        },
        ztrain_roll.index[:n_show],
    )
    # ytrain_pred.shape

In [ ]:
# test if model can predict on shuffled data
xtrain_shuffled = xtrain_roll.copy()
ytrain_shuffled = ytrain_roll.copy()
np.random.shuffle(xtrain_shuffled)
np.random.shuffle(ytrain_shuffled)
z_ind = ztrain_roll.reset_index().index.values
np.random.shuffle(z_ind)
ztrain_shuffled = ztrain_roll.iloc[z_ind]

n_show = 100
for n_start in [0,  50, 100, 150]:
    ytrain_pred = my_predict(
        mod2,
        {   'raw_features': xtrain_shuffled[n_start:(n_start+n_show)],
            'raw_targets':  ytrain_shuffled[n_start:(n_start+n_show)] + 1,
        },
        ztrain_shuffled.index[:n_show],
    )
    # ytrain_pred.shape

## save model

## plot trained result

In [ ]:
# re-build data without balancing and shuffling and with overlap
n_show = 100000
xtrain_ori, ytrain_ori, ztrain_ori = preprocess(x_train.head(n=n_show), y_train.head(n=n_show), with_overlap=True)
print(xtrain_ori.shape, ytrain_ori.shape, ztrain_ori.shape)

In [ ]:
xtrain_plot = pd.DataFrame(xtrain_ori[:,-1,:], columns=x_train.columns)
print(xtrain_plot.shape)
xtrain_plot[['Fp1', 'Fp2']].plot(figsize=(20,3), alpha=0.5)
# plt.title('subj_id=1, series_id=1')
plt.show()

In [ ]:
ytrain_plot = pd.DataFrame(ytrain_ori[:,-1,:], columns=y_train.columns)
print(ytrain_plot.shape)
ytrain_plot['HandStart'].plot(figsize=(20,3), alpha=0.5)
# plt.title('subj_id=1, series_id=1')
plt.show()

In [ ]:
ytrain_pred = my_predict(
    mod2,
    {   'raw_features': xtrain_ori,
        'raw_targets':  ytrain_ori + 1.1,
    },
    ztrain_ori.index,
)
# ytrain_pred.shape

In [ ]:
ytrain_pred.max()

## predict on test data

In [ ]:
n_test = train_features.shape[0] - n_train
x_test = train_features.tail(n=n_test).copy()
y_test = train_targets.tail(n=n_test).copy()
print('x_test, y_test', x_test.shape, y_test.shape)

xtest_roll, ytest_roll, ztest_roll = preprocess(x_test, y_test)
xtest_roll.shape, ytest_roll.shape, ztest_roll.shape

In [ ]:
n_show = 10000*5
ytest_pred = my_predict(
    mod2,
    {   'raw_features': xtest_roll[:n_show],
        'raw_targets':  ytest_roll[:n_show] + 1.1,
    },
    ztest_roll.index[:n_show],
)
ytest_pred.shape

## predict on new subject

In [ ]:
subj2_features, subj2_targets = my_load(subj_ids = [2], series_ids = [x+1 for x in range(8)])
subj2_features.shape, subj2_targets.shape

In [ ]:
x_subj2 = subj2_features.copy()
y_subj2 = subj2_targets.copy()
print('x_subj2, y_subj2', x_subj2.shape, y_subj2.shape)

xsubj2_roll, ysubj2_roll, zsubj2_roll = preprocess(x_subj2, y_subj2)
assert xsubj2_roll.shape[0] > 0
xsubj2_roll.shape, ysubj2_roll.shape, zsubj2_roll.shape

In [ ]:
n_show = 1000*5
ysubj2_pred = my_predict(
    mod2,
    {   'raw_features': xsubj2_roll[:n_show],
        'raw_targets':  ysubj2_roll[:n_show] + 1.1,
    },
    zsubj2_roll.index[:n_show],
)
ysubj2_pred.shape